In [ ]:
%pip install weaviate-client unstructured dspy openai ipywidgets anthropic cohere

In [8]:
# importando o que precisaremos
import dspy
import openai
import os
from unstructured.partition.html import partition_html

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dudanogueira/Library/Application Support/sagemaker/config.yaml


In [1]:
input_html = "https://thedevconf.com/tdc/2024/summit-brasilia/"
document_elements = partition_html(url=input_html, chunking_strategy="basic")

In [2]:
print(f"Total de chunks {len(document_elements)}")
for i in document_elements[25:30]:
    print("#")
    print(i.text)

Total de chunks 161
#
Prontidão Quântica: Como se preparar para o futuro da Computação?

23/10/2024 13:30 -14:05Palco 2
#
Nesta palestra serão tocados conceitos básicos da Computação Quântica, com destaque para as principais diferenças deste paradigma tecnológico em relação à Computação Clássica, bem como apresentado um panorama do cenário que se aproxima no mundo da Computação e dos Negócios com esta nova forma de resolução de problemas computacionais. Como se preparar para a Computação Quântica? Ela resolverá todos os nossos problemas? O que de fato muda? Quantum Machine Learning... O que é isto? Estas perguntas
#
e vivências do palestrante, enquanto especialista da área no Banco do Brasil, serão trazidas e discutidas.
#
Gustavo Botelho de Souza Banco do Brasil

Desconstruindo tarefas complexas com agentes inteligentes Evandro Franco (AWS)

14:15 -14:50

Desconstruindo tarefas complexas com agentes inteligentes

23/10/2024 14:15 -14:50Palco 2
#
Esta sessão explorará o fascinante mund

In [7]:
# vamos juntar todos os pedaços, como se fossem um texto contínuo
sample = "\n".join([i.text for i in document_elements])
print(sample[0:500])

TDC SUMMIT IA BRASÍLIA INTELIGÊNCIA ARTIFICIAL,LIDERANÇA TÉCNICA,GESTÃO e VOCÊ!

Quando?

23 e 24 de Outubro 2024 09:00 às 19:00 h GMT-3 HÍBRIDO: PRESENCIAL E ONLINE

Onde?

Centro de Convenções Ulysses Guimarães SDC - Ulysses Guimarães Brasília - DF Mais informações

Chamadas

Call4Trilhas: 21 de Maio a 24 de Junho Call4Papers: 21 de Maio a 25 de Julho Inscrições: 15 de Julho a 24 de Outubro Voluntários: Pré-inscrições: 01/10/2024

Inscrições
Fase 2: até 03 de Outubro Fase 3: 04 a 24 de Outubro


In [9]:
gpt4 = dspy.LM("openai/gpt-4o", max_tokens=4_000, model_type="chat")

In [10]:
openai.api_key = os.environ.get("OPENAI_APIKEY")
CLAUDE_API_KEY = os.environ.get("CLAUDE_APIKEY")
cohere_api_key = os.environ.get("COHERE_APIKEY")

# Start Llama 3.1 on your laptop with Ollama using!
'''bash
ollama run llama3.1
'''

gpt4 = dspy.LM("openai/gpt-4o", max_tokens=4_000, model_type="chat")
claude_opus = dspy.LM("anthropic/claude-3-opus-20240229", api_key=CLAUDE_API_KEY)
command_r_plus = dspy.LM("cohere/command-r-plus", max_tokens=4000, api_key=cohere_api_key)
ollama_llama3 = dspy.LM("ollama/llama3.1")

# você pode comentar abaixo a linha se não quiser testar todos
lms = [
    #{"name": "GPT-4", "lm": gpt4},
    #{"name": "Claude Opus", "lm": claude_opus},
    #{"name": "Command R+", "lm": command_r_plus},
    {"name": "Llama 3.1", "lm": ollama_llama3}
]

connection_prompt = "Fale algo interessante sobre sistemas de banco de dados para me impressionar com sua inteligência"


In [11]:
# vamos executar nosso multi prompt!

print(f"\033[36mTestando prompt:\n\033[91m\n{connection_prompt}\n")

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[92mResultado para {name}\n")
        print(f"\033[0m{lm(connection_prompt)[0]}\n")

Testando prompt:

Fale algo interessante sobre sistemas de banco de dados para me impressionar com sua inteligência

Resultado para Llama 3.1

Sistemas de banco de dados são fundamentais em muitas aplicações, desde pequenas bases de dados até grandes bancos de dados que armazenam informações de milhões de usuários. Aqui estão algumas características e conceitos interessantes sobre sistemas de banco de dados:

1. **Modelagem de Dados**: Um dos aspectos mais importantes é a modelagem de dados. Isso envolve descrever as entidades, relações e regras que governam os dados armazenados no banco de dados. Existem várias abordagens para isso, incluindo modelos relacionais (como o SQL), modelo de chave-valor (como o Redis) e modelos NoSQL mais flexíveis.

2. **Concorrência**: Com a crescente necessidade de escalabilidade em aplicações, os sistemas de banco de dados precisam lidar com concorrência. Isso significa permitir que múltiplos usuários ou processos acessem e modifiquem o banco de dados s

In [18]:
# Montando um Program de Chunking no dspy

from typing import List
from pydantic import BaseModel

class ChunkWithSummary(BaseModel):
    chunk: str
    summary: str
    category: str
    
class Chunker(dspy.Signature):
    """Your task is to divide a long document into coherent, semantic chunks of text. Each chunk should represent a complete thought or topic, typically ranging from one to three paragraphs in length. Follow these guidelines:
1. Focus on semantic coherence: Each chunk should contain text that revolves around a single main idea or closely related ideas.
2. Respect natural breaks: Use paragraph boundaries as a guide, but don't be bound by them if a topic continues across paragraphs.
3. Maintain context: Ensure that each chunk can be understood independently without losing crucial context.
4. Aim for consistency: Try to keep chunks relatively similar in length, but prioritize semantic completeness over strict length adherence.
5. Handle transitions: When encountering transition sentences between topics, include them with the most relevant chunk.
6. Consider document structure: Pay attention to headings, subheadings, and other structural elements that might indicate topic changes.
7. Adjust for content type: Be flexible based on the document's nature (e.g., academic papers might have longer chunks than news articles).
8. The content will have a lot of talks, with diffenret hours associated. Try to separate each talk on it's own chunk with date/time and author

Your output should be a List of strings in portuguese containing these semantic chunks, each representing a distinct portion of the original document while preserving its overall flow and meaning.
    """
    
    long_document: str = dspy.InputField()
    # file_summary: str = dspy.OutputField()
    chunks_with_summaries: List[ChunkWithSummary] = dspy.OutputField()
    
chunker = dspy.TypedPredictor(Chunker)

In [19]:
# vamos definir qual llm usar
# obs: este passo pode demorar alguns minutos...
dspy.settings.configure(lm=gpt4)
response = chunker(long_document=sample)

In [20]:
chunks_with_summaries = response.chunks_with_summaries

In [21]:
# temos agora menos objetos com melhor semântica!!
len(chunks_with_summaries)

10

In [22]:
print(f"Total de chunks {len(chunks_with_summaries)}")
for i, chunk in enumerate(chunks_with_summaries):
    print(f"\033[92m=== Chunk {i} ===\n\033[0m")
    print(f"{chunk.chunk}\n")
    print(f"\033[92m=== Sumário do Chunk {i} ===\n\033[0m")
    print(f"{chunk.summary}\n")
    print(f"{chunk.category}\n")

Total de chunks 10
=== Chunk 0 ===

TDC SUMMIT IA BRASÍLIA INTELIGÊNCIA ARTIFICIAL,LIDERANÇA TÉCNICA,GESTÃO e VOCÊ! Quando? 23 e 24 de Outubro 2024 09:00 às 19:00 h GMT-3 HÍBRIDO: PRESENCIAL E ONLINE Onde? Centro de Convenções Ulysses Guimarães SDC - Ulysses Guimarães Brasília - DF Mais informações Chamadas Call4Trilhas: 21 de Maio a 24 de Junho Call4Papers: 21 de Maio a 25 de Julho Inscrições: 15 de Julho a 24 de Outubro Voluntários: Pré-inscrições: 01/10/2024 Inscrições Fase 2: até 03 de Outubro Fase 3: 04 a 24 de Outubro Inscrições Individuais Inscrições Corporativas Inscrições Gratuitas Qual Ticket escolher?

=== Sumário do Chunk 0 ===

Informações gerais sobre o TDC Summit IA Brasília, incluindo datas, local, formato do evento, e detalhes sobre inscrições e chamadas para trilhas e papers.

Informações Gerais

=== Chunk 1 ===

O maior evento de tecnologia do BRASIL está chegando a Brasilia com uma edição especial dedicada à Inteligência Artificial. Serão três palcos repletos de pal

In [23]:
# Se usarmos um método de chunking "comum", 
# por quantidades máximas e sobreposição, o resultado seria esse:
from unstructured.partition.html import partition_html
input_html = "https://thedevconf.com/tdc/2024/summit-brasilia/"
document_elements = partition_html(url=input_html, chunking_strategy="basic")

print("TOTAL DE CHUNKS", len(document_elements))

for i in document_elements[0:5]:
    print("#####")
    print(i.text)

TOTAL DE CHUNKS 162
#####
TDC SUMMIT IA BRASÍLIA INTELIGÊNCIA ARTIFICIAL,LIDERANÇA TÉCNICA,GESTÃO e VOCÊ!

Quando?

23 e 24 de Outubro 2024 09:00 às 19:00 h GMT-3 HÍBRIDO: PRESENCIAL E ONLINE

Onde?

Centro de Convenções Ulysses Guimarães SDC - Ulysses Guimarães Brasília - DF Mais informações

Chamadas

Call4Trilhas: 21 de Maio a 24 de Junho Call4Papers: 21 de Maio a 25 de Julho Inscrições: 15 de Julho a 24 de Outubro Voluntários: Pré-inscrições: 01/10/2024

Inscrições
#####
Fase 2: até 03 de Outubro Fase 3: 04 a 24 de Outubro

Inscrições Individuais

Inscrições Corporativas

Inscrições Gratuitas

Qual Ticket escolher?

O maior evento de tecnologia do BRASIL está chegando a Brasilia com uma edição especial dedicada à Inteligência Artificial.
#####
Serão três palcos repletos de palestras técnicas, painéis de discussão, cases de sucesso e sessões focadas em processos e desenvolvimento. Além disso, teremos workshops práticos para quem deseja aprender de forma prática. O evento também trar

In [24]:
# agora vamos salvar os resultados em nosso banco de dados
# lembre-se de fechar a conexão no exemplo-basico.ipynb!!!

import os
import weaviate
from weaviate import classes as wvc

# aqui vamos usar a versão embedded, para testes/desenvolvimento, que roda direto do cliente
# o recomendado para produção é docker ou kubernetes
client = weaviate.connect_to_embedded(
    version="1.26.7",
    headers={
        "X-Cohere-Api-Key": os.environ.get("COHERE_APIKEY")
    }
)
# vamos mostra a versão do cliente e do servidor
print(weaviate.__version__, client.get_meta().get("version"))

{"action":"startup","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-10-21T10:17:48-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-10-21T10:17:48-03:00"}
{"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-10-21T10:17:48-03:00"}
{"build_git_commit":"fdb81e790","build_go_versio

4.9.0 1.26.7


{"action":"lsm_recover_from_active_wal","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","class":"Palestras","index":"palestras","level":"warning","msg":"empty write-ahead-log found. Did weaviate crash prior to this or the tenant on/loaded from the cloud? Nothing to recover from this file.","path":"/Users/dudanogueira/.local/share/weaviate/palestras/et3QcOOVS3xQ/lsm/property_chunk_summary/segment-1729281981333564000","shard":"et3QcOOVS3xQ","time":"2024-10-21T10:17:50-03:00"}
{"action":"lsm_recover_from_active_wal","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","class":"Palestras","index":"palestras","level":"warning","msg":"empty write-ahead-log found. Did weaviate crash prior to this or the tenant on/loaded from the cloud? Nothing to recover from this file.","path":"/Users/dudanogueira/.local/share/weaviate/palestras/et3QcOOVS3xQ/lsm/property__i

In [26]:
# vamos garantir que não temos a coleção "Palestras"
client.collections.delete("Palestras")
# vamos criar a coleção de palestras
collection = client.collections.create(
    name="Palestras",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_cohere(),
    generative_config=wvc.config.Configure.Generative.cohere(),
    properties=[
        wvc.config.Property(name="content", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="chunk_summary", data_type=wvc.config.DataType.TEXT),
    ]
)

{"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/palestras/4fpgJePvTcpZ/proplengths does not exist, creating new tracker","time":"2024-10-21T10:18:27-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-10-21T10:18:27-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"Created shard palestras_4fpgJePvTcpZ in 2.651291ms","time":"2024-10-21T10:18:27-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version

In [27]:
for data_row in chunks_with_summaries:
    print({"content": str(data_row.chunk), "chunk_summary": str(data_row.summary)})

{'content': 'TDC SUMMIT IA BRASÍLIA INTELIGÊNCIA ARTIFICIAL,LIDERANÇA TÉCNICA,GESTÃO e VOCÊ! Quando? 23 e 24 de Outubro 2024 09:00 às 19:00 h GMT-3 HÍBRIDO: PRESENCIAL E ONLINE Onde? Centro de Convenções Ulysses Guimarães SDC - Ulysses Guimarães Brasília - DF Mais informações Chamadas Call4Trilhas: 21 de Maio a 24 de Junho Call4Papers: 21 de Maio a 25 de Julho Inscrições: 15 de Julho a 24 de Outubro Voluntários: Pré-inscrições: 01/10/2024 Inscrições Fase 2: até 03 de Outubro Fase 3: 04 a 24 de Outubro Inscrições Individuais Inscrições Corporativas Inscrições Gratuitas Qual Ticket escolher?', 'chunk_summary': 'Informações gerais sobre o TDC Summit IA Brasília, incluindo datas, local, formato do evento, e detalhes sobre inscrições e chamadas para trilhas e papers.'}
{'content': 'O maior evento de tecnologia do BRASIL está chegando a Brasilia com uma edição especial dedicada à Inteligência Artificial. Serão três palcos repletos de palestras técnicas, painéis de discussão, cases de sucesso

In [28]:
# agora vamos importar todos os nossos chunks semânticos

with collection.batch.dynamic() as batch:
    for data_row in chunks_with_summaries:
        batch.add_object(
            properties={
                "content": data_row.chunk, 
                "chunk_summary": data_row.summary
            }
        )
if collection.batch.failed_objects:
    print(collection.batch.failed_objects)
else:
    print("Batch import executed without errors")

Batch import executed without errors


In [29]:
collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=10)

In [30]:
for obj in collection.query.near_text("Palestra sobre proteção e invasão", auto_limit=1).objects:
    print("####")
    print(obj.properties)

####
{'content': 'Superfice de Ataque: Como minimizar riscos em um Mundo Digital. Paulo Lamellas (TechSecurity Consultoria) 16:20 -16:55 Na era atual, ataques cibernéticos são uma grande preocupação para organizações de todos os tamanhos. Uma das maneiras mais eficazes de mitigar esses riscos é gerenciando a superfície de ataque. Esta apresentação explora a importância de gerenciar a superfície de ataque e as estratégias que podemos usar para reduzir o risco de ciberataques.', 'chunk_summary': 'Paulo Lamellas aborda a gestão da superfície de ataque para minimizar riscos cibernéticos, discutindo estratégias de segurança.'}
####
{'content': '23/10/2024 Palco 1 Abertura Yara Mascarenhas Hornos (TDC) Bruno Souza (Code4.Life) 09:00 -09:30 Keynote Plano Brasileiro de Inteligência Artificial Cristina Shimoda (Ministério da Ciência, Tecnologia e Inovação) 09:40 -10:15 Oportunidades a partir do Plano Brasileiro de Inteligência Artificial Cristina Shimoda Ministério da Ciência, Tecnologia e Inov

In [31]:
# vamos fechar o cliente
client.close()

{"action":"restapi_management","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","docker_image_tag":"localhost","level":"info","msg":"Shutting down... ","time":"2024-10-21T10:18:53-03:00"}
{"action":"restapi_management","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","docker_image_tag":"localhost","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-10-21T10:18:53-03:00"}
{"action":"telemetry_push","build_git_commit":"fdb81e790","build_go_version":"go1.23.2","build_image_tag":"localhost","build_wv_version":"1.26.7","level":"info","msg":"telemetry terminated","payload":"\u0026{MachineID:119b550e-c485-46ef-b99e-71974cf6f9e5 Type:TERMINATE Version:1.26.7 NumObjects:4 OS:darwin Arch:arm64 UsedModules:[generative-cohere text2vec-cohere]}","time":"2024-10-21T10:18:53-03:00"}
{"build_git_commit":"fdb81e790","build_go_versi